In [7]:
import numpy as np

result = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100409', 'score': 0.875}}}, 'date_time': '02/15/2022, 15:35:24', 'label_id': 157, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0157.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 180, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result1 = {'barcode': '2500000279935', 'cert_mark': ['organic'], 'cert_result': {'04829818': {'in_db': False, 'mark_status': 'success', 'number': {'db': '04829818', 'ocr_rslt': '04829818'}}, '12100489': {'in_db': True, 'mark_status': 'success', 'name': {'db': '김영대', 'ocr_rslt': '김영대', 'score': 1.0}, 'number': {'db': '12100489', 'ocr_rslt': '12100489', 'score': 1.0}}}, 'date_time': '02/15/2022, 16:40:53', 'label_id': 562, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000279935/0562.png', 'product_name': {'name': '친환경 방울토마토', 'status': 'success'}, 'rot_angle': 0, 'weight': {'db': '600g', 'ocr_rslt': '600g', 'score': 1.0, 'status': 'success'}}

result2 = {'barcode': '2500000145629', 'cert_mark': ['organic'], 'cert_result': {}, 'date_time': '02/17/2022, 15:53:04', 'label_id': 142, 'label_loc': '/mnt/vitasoft/salim/detected_labels/2500000145629/0142.png', 'product_name': {'name': '유기농 표고버섯', 'status': 'success'}, 'rot_angle': 90, 'weight': {'db': '300g', 'ocr_rslt': '300g', 'score': 1.0, 'status': 'success'}}

mark_kor = {
    'organic': '유기농',
    'nonpesticide': '무농약',
    'gap': 'GAP(우수관리인증)',
    'antibiotic': '무항생제',
    'animal': '동물복지',
    'haccp': '안전관리인증HACCP',
    'pgi': '지리적표시',
    'traditional': '전통식품',
    'master': '식품명인',
    'processed': '가공식품',
    'carbon': '저탄소(LOW CARBON)'
}

def inspection_result(result):
    return_dict = {'product_name': [], 'weight': [], 'barcode': [], 'cert_mark': [], 'cert_result': []}
    for key, value in result.items():
        cert_third_num = []
        if key == 'product_name':
            return_dict[key].append(value['name'])
            return_dict[key].append(value['name'])
            if value['status'] == 'success':
                return_dict[key].append('100%')
                return_dict[key].append('승인')

        if key == 'weight':
            return_dict[key].append(value['ocr_rslt'])
            return_dict[key].append(value['db'])
            return_dict[key].append(f"{round(value['score'] * 100)}%")
            return_dict[key].append('승인')

        if key == 'barcode':
            return_dict[key].append(value)
            return_dict[key].append(value)
            return_dict[key].append('100%')
            return_dict[key].append('승인')

        if key == 'cert_mark':
            if value:
                for mark in value:
                    mark_lst = []
                    mark_lst.append(mark_kor[mark])
                    cert_numbers = [item['number']['ocr_rslt'] for item in result['cert_result'].values()]
                    mark_lst.append(', '.join([number[2] for number in cert_numbers]))
                    mark_status = [number['mark_status'] for number in result['cert_result'].values()]
                    count = 0
                    for status in mark_status:
                        if 'success' == status:
                            count += 1
                    if len(mark_status) > 0:
                        raw_score = count / len(mark_status)
                    else:
                        raw_score = 0.0
                    score = f"{round(raw_score * 100)}%"
                    mark_lst.append(score)

                    if count == 2 and raw_score >= 0.9:
                        mark_lst.append('승인')
                    elif 'fail' in mark_status:
                        mark_lst.append('오류')
                    else:
                        mark_lst.append('매칭실패')
                    return_dict[key].append(mark_lst)
            else:
                mark_lst = ['', '', '0.0']
                return_dict[key].append(mark_lst)

        if key == 'cert_result':
            for num_key, num_value in value.items():
                cert_lst = []
                if num_value['in_db']:
                    cert_lst.append(f"{num_value['number']['ocr_rslt']} {num_value['name']['ocr_rslt']}")
                    cert_lst.append(f"{num_value['number']['db']} {num_value['name']['db']}")
                    raw_score = (num_value['name']['score'] + num_value['number']['score']) / 2.0
                    cert_lst.append(f"{round(raw_score * 100)}%")
                    result = '오류'
                    if raw_score >= 0.9:
                        result = '승인'
                    cert_lst.append(result)
                else:
                    name = ''
                    number = ''
                    if 'name' in num_value and 'number' in num_value:
                        name = num_value['name']['ocr_rslt']
                        number = num_value['number']['ocr_rslt']
                    elif 'number' in num_value.keys():
                        number = num_value['number']['ocr_rslt']
                    elif 'name' in num_value.keys():
                        name = num_value['name']['ocr_rslt']
                    cert_lst.append(f"{name} {number}")
                    cert_lst.append("매칭실패")
                    cert_lst.append('0%')
                    cert_lst.append('매칭실패')
                return_dict[key].append(cert_lst)

    return return_dict

In [23]:
from glob import glob
import os
result_dict = inspection_result(result2)
print(result_dict['product_name'][1])
root_path = 'A:/salim/detected_labels'
len(glob(os.path.join(root_path, str(result_dict['barcode'][0]), '*.png')))

유기농 표고버섯


255

In [46]:
[{'center': [3342, 515], 'label_id': 39, 'label_loc': '/mnt/vitasoft/salim/detected_labels/unrecognized/0039.png', 'detected_mark': ['organic'], 'barcode_result': {'status': 'barcode_not_found'}, 'producer_result': {'12100779': {'in_db': False, 'number': {'db': '12100779', 'ocr_rslt': '12100779'}, 'mark_status': 'success'}}, 'rot_angle': 90}]

TypeError: isna() takes 1 positional argument but 2 were given

In [214]:
import pandas as pd
class Db():
    def __init__(self):
        self.df = pd.read_excel('../../Database/제품등록정보20211015.xlsx')
        self.df = self.df[self.df['라벨제품명'].str.contains('운영중단') == False]
        self.df = self.df.fillna('')

    def get_product_names(self):
        lst = ['라벨제품명', '중량(수량)', '단위']
        df = self.df[lst].values.tolist()
        product_names = []
        for idx in df:
            product_names.append(''.join([str(word) for word in idx]))
        return product_names

    def get_product_name_by_barcode(self, barcode):
        df = self.df
        df = df[df['바코드'].astype(float) == barcode]
        product_name = df['라벨제품명'].values.tolist()
        return product_name[0]

    def get_product_nick_names(self):
        nick_names = list(self.df['라벨제품명별칭'])
        return nick_names

    def get_product_barcodes(self):
        barcodes = list(self.df['바코드'])
        return barcodes

    def get_product_inspections(self):
        lst = ['제품명검사', '중량(입수)검사', '인증마크검사', '인증정보검사']
        inspections = self.df[lst]
        inspections = inspections.replace('검사', 'O')
        inspections = inspections.replace('pass', 'X')
        print(type(inspections))
        inspections_lst = inspections.values.tolist()
        barcodes = self.get_product_barcodes()
        check_dict = {}
        for idx, barcode in enumerate(barcodes):
            check_dict[barcode] = inspections_lst[idx]
        return check_dict

    def product_inspection_result(self):
        product_names = self.get_product_names()
        result_dict = {}
        for product_name in product_names:
            count = {'누적불합격': 0, '연속유보': 0, '총인식수': 0}
            result_dict[product_name] = count
        return result_dict

In [217]:
db = Db()

In [218]:
db.get_product_names()
db.get_product_name_by_barcode(2500000279935)

'친환경 방울토마토'

In [196]:

import pandas as pd
import numpy as np
df = pd.read_excel('../../Database/제품등록정보20211015.xlsx')
df = df[df['라벨제품명'].str.contains('운영중단') == False]
df = df.fillna('')
df = df[df['바코드'].astype(float) == 2500000082634]
product_name = df['라벨제품명']
product_name.values.tolist()

['유기농 깐 밤']

In [111]:


# df[df['바코드'].str.contains(2500000279935) == True]
df[df['바코드'].astype(float) == 2500000279935]
df['바코드']

,ERP코드,ERP품목명,바코드,라벨제품명,라벨제품명별칭,중량(수량),단위,제품패턴검사,제품명검사,중량(입수)검사,바코드검사,인증마크검사,인증정보검사,Unnamed: 13
14,5352,친환경 방울토마토 600g,2500000279935,친환경 방울토마토,유기농 방울토마토,600,g,검사,pass,pass,pass,pass,pass,


In [ ]:

# df = self.db.get_product_inspections()
# df = df.replace('검사', 'O')
# df = df.replace('pass', 'X')
# for idx in range(len(df)):
#     check_lst.append(list(df.iloc[idx].to_dict().values()))
# barcode_lst = list(df['바코드'].to_dict().values())
# for idx, barcode in enumerate(barcode_lst):
#     check_inspection_daily[barcode] = check_lst[idx]


# db = Db()
# db.product_inspection_result()

In [95]:
df = pd.read_excel('../../Database/제품등록정보20211015.xlsx')
df = df[df['라벨제품명'].str.contains('운영중단') == False]
df = df.fillna('')
lst = ['제품명검사', '중량(입수)검사', '인증마크검사', '인증정보검사']
inspections = df[lst]
inspections = inspections.replace('검사', 'O')
inspections = inspections.replace('pass', 'X')
inspections_lst = inspections.values.tolist()
db = Db()
barcodes = db.get_product_barcodes()
check_dict = {}
for idx, barcode in enumerate(barcodes):
    check_dict[barcode] = inspections_lst[idx]
check_dict

{2500000082634: ['O', 'O', 'O', 'O'],
 2500000070556: ['O', 'O', 'O', 'O'],
 2500000097904: ['O', 'O', 'O', 'O'],
 1500000031541: ['O', 'O', 'O', 'O'],
 2500000100123: ['O', 'O', 'O', 'O'],
 1500000002428: ['O', 'O', 'O', 'O'],
 1500000119546: ['O', 'O', 'X', 'X'],
 1500000157944: ['O', 'O', 'O', 'O'],
 1500000002466: ['O', 'O', 'O', 'O'],
 2500000245466: ['X', 'X', 'X', 'X'],
 2500000178498: ['X', 'X', 'X', 'X'],
 2500000252983: ['X', 'X', 'X', 'X'],
 2500000145629: ['O', 'O', 'X', 'O'],
 2500000279935: ['O', 'O', 'O', 'O'],
 2500000279942: ['O', 'O', 'O', 'O'],
 1500000050207: ['O', 'O', 'O', 'O'],
 2500000119200: ['O', 'O', 'O', 'O'],
 2500000021893: ['O', 'O', 'O', 'O'],
 2500000289552: ['O', 'O', 'O', 'O'],
 2500000292446: ['O', 'O', 'O', 'O'],
 2500000296963: ['O', 'O', 'O', 'O'],
 2500000301681: ['O', 'O', 'X', 'X']}

In [1]:
result_dict = {'product_name': ['유기농 표고버섯', '유기농 표고버섯', '100%', '승인'], 'weight': ['300g', '300g', '100%', '승인'], 'barcode': ['2500000145629', '2500000145629', '100%', '승인'], 'cert_mark': [], 'cert_result': [['12100179 금사행버섯문과', '12100779 흙사랑버섯분과', '82%', '오류']]}

result_dict

{'product_name': ['유기농 표고버섯', '유기농 표고버섯', '100%', '승인'],
 'weight': ['300g', '300g', '100%', '승인'],
 'barcode': ['2500000145629', '2500000145629', '100%', '승인'],
 'cert_mark': [],
 'cert_result': [['12100179 금사행버섯문과', '12100779 흙사랑버섯분과', '82%', '오류']]}